### 5.5 계층적 색인

계층적 색인(Hierarchical indexing)은 고차원 데이터를 낮은 차원의 형식으로 다룰 수 있게 합니다.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
data = Series(np.random.randn(10), index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
             [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1    0.282909
   2   -0.380583
   3   -1.493881
b  1    1.016437
   2   -0.436510
   3    1.887517
c  1    0.115373
   2   -0.867160
d  2   -0.079844
   3   -1.220262
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1    1.016437
2   -0.436510
3    1.887517
dtype: float64

In [5]:
data['b':'c']

b  1    1.016437
   2   -0.436510
   3    1.887517
c  1    0.115373
   2   -0.867160
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1    1.016437
   2   -0.436510
   3    1.887517
d  2   -0.079844
   3   -1.220262
dtype: float64

In [7]:
data[:, 2]

a   -0.380583
b   -0.436510
c   -0.867160
d   -0.079844
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,0.282909,-0.380583,-1.493881
b,1.016437,-0.436510,1.887517
c,0.115373,-0.867160,NaN
d,NaN,-0.079844,-1.220262


In [9]:
data.unstack().stack()

a  1    0.282909
   2   -0.380583
   3   -1.493881
b  1    1.016437
   2   -0.436510
   3    1.887517
c  1    0.115373
   2   -0.867160
d  2   -0.079844
   3   -1.220262
dtype: float64

In [10]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=[['Ohio', 'Ohio', 'Colorado'],
                         ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [17]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

In [18]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                     names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#### 5.5.1 계층 순서 바꾸고 정렬하기

계층 순서를 바꾸거나 지정된 계층에 따라 데이터를 정렬해야 한는 경우

In [19]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [26]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [28]:
frame.sort_index(1)
# frame.sortlevel 사용대신 sort_index 나 sort_value 메서드 사용으로 바뀌었다.

state     Colorado  Ohio    
color        Green Green Red
key1 key2                   
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10

In [23]:
frame.swaplevel(0, 1).sort_index(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### 5.5.2 단계별 요약통계

In [29]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [32]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### 5.5.3 DataFrame 칼럼 사용하기

In [33]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                  'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                  'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


위같은 DataFrame 이 있을 때 c, d를 인덱스로 하는 DataFrame을 만들어보자

In [38]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [40]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [41]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3